TUGAS BESAR INTELEGENSI BUATAN - RB

NAMA KELOMPOK :
1. Bayu Agaluh Wijaya        121140097
2. Fadhil Firoos             121140142
3. Ilham Yoga Pratama        121140081
4. Andrean Syahrezi          121140169
5. Maharani Triza Putri      121140071

In [ ]:
# import library
import string
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from util import JSONParser

### Load Data

Data yang digunakan merupakan data JSON. Tujuannya adalah mengubah bentuk JSON menjadi dalam bentuk tabel agar mudah ditraining menggunakan *scikit-learn*.
Kami sudah membuat parser khusus untuk data dengan format tersebut didalam direktori `util/` yang bernama `JSONParser`

In [ ]:
# load data
path = "data/intents.json"

# buat objek JSONParser dan parse data intents.json
jp = JSONParser()
jp.parse(path)

# simpan dataframe dalam variabel df
df = jp.get_dataframe()

In [ ]:
# lihat 5 data pertama
df.head()

In [ ]:
# hitung jumlah data per tag / inten
df.intents.value_counts()

### Praproses Data

Pada bagian ini data training akan dibersihkan dan dilakukan praproses apabila diperlukan.

Praproses yang dilakukan adalah :
- Case Folding : Mengubah semua alfabet menjadi huruf kecil
- Menghapus tanda baca


In [ ]:
def preprocess(chat):
    """
    Fungsi yang digunakan untuk melakukan praproses
    """
    # konversi ke lowercase
    chat = chat.lower()
    # menghapus tanda baca
    tandabaca = tuple(string.punctuation)
    chat = ''.join(ch for ch in chat if ch not in tandabaca)
    return chat

In [ ]:
# implementasikan fungsi preprocess ke string
df['text_input_prep'] = df.text_input.apply(preprocess)

Apabila kita lihat hasilnya maka kita dapati hal berikut :

In [ ]:
df[['text_input', 'text_input_prep']].head(10)

Kita dapat lihat bahwa kolom kiri merupakan kolom data sebelum di praproses sedangkan yang kanan adalah input yang sudah di praproses

### Vektorisasi

Sebelum kita masukkan data kita kedalam algoritma machine learning, kita perlu melakukan vektorisasi terlebih dahulu. Karena input yang diterima oleh model machine learning harus berupa matriks sedangkan dataset yang kita proses berupa teks.

Metode vektorisasi paling sederhana adalah metode *bag of words* yaitu metode pengumpulan *vocab* yang terdapat pada *corpus*.
Metode *bag of words* yang paling sederhana adalah metode `CountVectorizer` yang terdapat pada *scikit-learn*.

In [ ]:
# inisiasi objek CountVectorizer
vect = CountVectorizer()

Langkah penting dalam metode *bag of words* adalah mengumpulkan *vocab* yang terdapat pada *corpus* yang kita miliki. Dalam scikit-learn kita dapat lakukan dengan cara :

In [ ]:
# mengumpulkan vocab dari data teks yang sudah dilakukan praproses
vect.fit(df['text_input_prep'])

In [ ]:
# Lihat list vocab
feature_names = vect.get_feature_names_out()[:10]  # batasi hanya 10 vocab teratas

# Tampilkan hasil
print(feature_names)

Selanjutnya konversi data teks menjadi matriks sesuai vocab yang sudah dibuat

In [ ]:
# ubah data teks menjadi matriks
text_vect = vect.transform(df.text_input_prep)

text_vect

Kita bisa lihat bahwa data teks kita sudah berubah menjadi bentuk *sparse matrix*

In [ ]:
# Lihat list vocab
feature_names = vect.get_feature_names_out()[:10]  # batasi hanya 10 vocab teratas

# Tampilkan hasil
print(feature_names)

Jika kita menggunakan dengan data yang besar, kita tidak disarankan melihat representasi matriks datanya karena akan membuat komputer menjadi berat

### Modelling

Setelah data siap, saatnya ditrain kedalam algoritma machine learning.
Dalam kasus ini kita menggunakan Multinomial Naive Bayes

In [ ]:
# deklarasi objek MultinomialNB
nb = MultinomialNB()

# training data, dengan X : text_vect dan y : intents
nb.fit(text_vect, df.intents)

Setelah model dilatih kita coba keluarkan hasil prediksi dari suatu input

In [ ]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict(chat)

# tampilkan hasil prediksi
print(f"Hasil prediksi : {res[0]}")

Supaya hasil menjadi lebih natural, kita lakukan prediksi dengan probabilitas sehingga apabila hasil prediksi kurang dari threshold probabilitas yang ditentukan, kita bisa buat bot untuk bilang "tidak mengerti"

In [ ]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict_proba(chat)

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

### Efisiensi dengan Pipeline

Dari proses diatas kita bisa lihat apabila ada data teks maka kita perlu proses dalam dua langkah, yaitu vektorisasi dan pemodelan. Supaya proses menjadi lebih ringkas dan lebih mudah dalam proses deployment, kita akan buat pipeline

In [ ]:
# Deklarasi pipeline yang mengandung vektorisasi (CountVectorizer) & pemodelan (MultinomialNB)
pipe = make_pipeline(CountVectorizer(),
                     MultinomialNB())

# Training
pipe.fit(df.text_input, df.intents)

Dapat kita lihat dari proses training diatas, seolah-olah kita langsung memasukkan data teks dan labelnya langsung kedalam "*black box*" sehingga proses prediksi akan lebih ringkas.

Untuk inference dengan pipeline dapat kita lakukan dengan cara berikut :

In [ ]:
# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# prediksi teks kedalam pipeline
res = pipe.predict_proba([chat])

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")

Dapat kita lihat bahwa teks baru setelah praproses bisa langsung masuk kedalam pipeline

### Simulasi Inference

Selanjutnya kita akan simulasikan chatbot mulai dari mendapatkan input sampai ke respon

Dalam kasus ini apabila intent yang terdeteksi adalah `bye` maka program berhenti

In [18]:
print("Anda Terhubung dengan chatbot Kami")
while True:
    # input user
    chat = input("Anda : ")
    # praproses
    chat = preprocess(chat)
    # prediksi intent
    res = pipe.predict_proba([chat])
    # ambil nilai probabilitas & lokasinya
    max_prob = max(res[0])
    max_idx = np.argmax(res[0])
    # kondisi jika probabilitas kurang dari threshold
    if max_prob < 0.20:
        print("Bot : Maaf Kak, aku ga ngerti")
    else:
        print(f"Bot : {jp.get_response(nb.classes_[max_idx])}")
    if nb.classes_[max_idx] == 'bye':
        break


Bot : Halo!
Bot : Aku bantu Kelompok IB dari Fadhil, Bayu, Ilham, Andreyan, Maharani dalam UAS IB
Bot : Kakak bisa tanya-tanya nama aku, apa yang aku lakukan dan yang berhubungan dengan pengenalan python
Bot : python kaak, jangan bikin emosi deh
Bot : python kaak, jangan bikin emosi deh
Bot : Python adalah bahasa pemrograman tingkat tinggi yang bersifat interpretatif, mudah dibaca, dan mendukung paradigma pemrograman berorientasi objek, prosedural, serta fungsional. Dikembangkan oleh Guido van Rossum pada awal 1990-an, Python menjadi populer karena sintaksisnya yang sederhana dan ekosistemnya yang kaya. Python digunakan dalam berbagai bidang seperti pengembangan web, ilmu data, kecerdasan buatan, pengembangan perangkat lunak, dan otomatisasi tugas. Kelebihan Python termasuk keberlanjutan, portabilitas, dan komunitas pengguna yang besar, yang berkontribusi pada pertumbuhan dan keberlanjutan bahasa ini.
Bot : Sampai jumpa lagi yaa


Setelah kita berhasil simulasikan bot nya dalam notebook ini, kita simpan modelnya agar dapat dideploy dengan mudah

In [19]:
with open("chatbot_pipeline.pkl", "wb") as model_file:
    pickle.dump(pipe, model_file)